In [2]:
# cmds 파일 업데이트 전용 스크립트 (sourceId 매칭 수정 버전)
import json
import re
from pathlib import Path

# ─────────────────────────────────────────────────────────
# 설정
# ─────────────────────────────────────────────────────────
NEED_CRAWLING_JSON = "need_crawling.json"
CRAWL_RESULT_JSON = "crawl_result_temp.json"
CMDS_INPUT = r"C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\2020\documents_2020_01.cmds"
CMDS_OUTPUT = "documents_2020_actTimefixed.cmds"

# ─────────────────────────────────────────────────────────
# 유틸 함수
# ─────────────────────────────────────────────────────────
def extract_doc_number(source_id):
    """sourceId에서 문서 번호만 추출
    예: "doc_2009502_01" → "2009502"
    """
    # 숫자만 추출
    numbers = re.findall(r'\d+', source_id)
    if not numbers:
        return source_id
    
    # 가장 긴 숫자 반환 (문서번호가 제일 김)
    return max(numbers, key=len)

# ─────────────────────────────────────────────────────────
# 1) 데이터 로드
# ─────────────────────────────────────────────────────────
print("="*70)
print("📝 cmds 파일 업데이트 시작")
print("="*70)

# need_crawling.json 로드
try:
    with open(NEED_CRAWLING_JSON, 'r', encoding='utf-8') as f:
        need_crawling = json.load(f)
    print(f"✅ {NEED_CRAWLING_JSON} 로드: {len(need_crawling)}개 문서")
except Exception as e:
    print(f"❌ {NEED_CRAWLING_JSON} 읽기 실패: {e}")
    exit(1)

# crawl_result_temp.json 로드
try:
    with open(CRAWL_RESULT_JSON, 'r', encoding='utf-8') as f:
        crawl_results = json.load(f)
    print(f"✅ {CRAWL_RESULT_JSON} 로드: {len(crawl_results)}개 문서")
except Exception as e:
    print(f"❌ {CRAWL_RESULT_JSON} 읽기 실패: {e}")
    exit(1)

# cmds 파일 확인
if not Path(CMDS_INPUT).exists():
    print(f"❌ {CMDS_INPUT} 파일이 없습니다!")
    exit(1)

print(f"✅ {CMDS_INPUT} 파일 확인\n")

# ─────────────────────────────────────────────────────────
# 2) cmds 파일 업데이트
# ─────────────────────────────────────────────────────────
print("="*70)
print("🔄 업데이트 처리 중...")
print("="*70)

updated_count = 0
total_lines = 0
matched_docs = 0

with open(CMDS_INPUT, 'r', encoding='utf-8') as f_in, \
     open(CMDS_OUTPUT, 'w', encoding='utf-8') as f_out:
    
    for line in f_in:
        total_lines += 1
        line = line.strip()
        
        if not line.startswith("addDocument "):
            f_out.write(line + '\n')
            continue
        
        # JSON 파싱
        try:
            json_str = line[12:]  # "addDocument " 제거
            doc = json.loads(json_str)
            source_id_raw = doc.get("sourceId")  # "doc_2009502_01"
            
            # 📌 숫자만 추출 (매칭용)
            source_id_num = extract_doc_number(source_id_raw)  # "2009502"
            
            # 크롤링 결과에 있는지 확인
            if source_id_num not in crawl_results:
                f_out.write(line + '\n')
                continue
            
            # 🔍 매치됨!
            matched_docs += 1
            print(f"\n  🔍 매치된 문서: {source_id_raw} → {source_id_num} (line {total_lines})")
            
            # activities 업데이트
            crawled = crawl_results[source_id_num]
            need_update = need_crawling.get(source_id_num, [])
            activities = doc.get("activities", [])
            
            print(f"    - need_update 항목 수: {len(need_update)}")
            print(f"    - activities 항목 수: {len(activities)}")
            print(f"    - crawled 항목 수: {len(crawled)}")
            
            for update_info in need_update:
                target_index = update_info["index"]
                target_name = update_info["name"]
                target_action = update_info["action_type"]
                
                print(f"\n    🎯 업데이트 시도: index={target_index}, name={target_name}, action={target_action}")
                
                # activities에서 매칭되는 항목 찾기
                if target_index >= len(activities):
                    print(f"      ❌ index {target_index}가 범위 초과 (activities 길이: {len(activities)})")
                    continue
                
                activity = activities[target_index]
                
                # index, name, action_type 모두 확인
                activity_name = activity.get("name", "")
                activity_type = activity.get("type", "")
                
                print(f"      - activities[{target_index}]: name={activity_name}, type={activity_type}")
                print(f"      - 매칭 체크: '{target_name}' in '{activity_name}' = {target_name in activity_name}")
                print(f"      - 매칭 체크: '{target_action}' == '{activity_type}' = {target_action == activity_type}")
                
                if target_name in activity_name and target_action == activity_type:
                    print(f"      ✅ 첫 번째 매칭 성공!")
                    
                    # 크롤링 결과에서 해당 항목 찾기
                    found = False
                    for crawled_item in crawled:
                        print(f"        - crawled[{crawled_item['index']}]: name={crawled_item['name']}, action={crawled_item['action_type']}")
                        
                        if (crawled_item["index"] == target_index and 
                            crawled_item["name"] in activity_name and
                            crawled_item["action_type"] == activity_type):
                            
                            # actionDate 업데이트
                            old_date = activity.get("actionDate")
                            activity["actionDate"] = crawled_item["timestamp"]
                            print(f"        ✅ 업데이트: {old_date} → {crawled_item['timestamp']}")
                            updated_count += 1
                            found = True
                            break
                    
                    if not found:
                        print(f"      ⚠️ crawled에서 매칭 항목 못 찾음")
                else:
                    print(f"      ❌ 첫 번째 매칭 실패")
            
            # ⚠️ 중요: 원본 sourceId는 그대로 유지!
            # doc["sourceId"]는 여전히 "doc_2009502_01"
            updated_line = "addDocument " + json.dumps(doc, ensure_ascii=False, separators=(',', ':'))
            f_out.write(updated_line + '\n')
            
        except Exception as e:
            print(f"  ⚠️ 파싱 에러 (line {total_lines}): {e}")
            f_out.write(line + '\n')

print("\n" + "="*70)
print("🎉 업데이트 완료!")
print("="*70)
print(f"📊 처리 결과:")
print(f"  - 전체 문서: {total_lines}개")
print(f"  - sourceId 매치: {matched_docs}개")
print(f"  - 업데이트: {updated_count}개 결재선")
print(f"  - 출력 파일: {CMDS_OUTPUT}")
print("="*70)

📝 cmds 파일 업데이트 시작
✅ need_crawling.json 로드: 171개 문서
✅ crawl_result_temp.json 로드: 171개 문서
✅ C:\Users\LEEJUHWAN\Desktop\애니파이브\전자결재\2020\documents_2020_01.cmds 파일 확인

🔄 업데이트 처리 중...

  🔍 매치된 문서: doc_2009497_01 → 2009497 (line 1)
    - need_update 항목 수: 3
    - activities 항목 수: 3
    - crawled 항목 수: 3

    🎯 업데이트 시도: index=0, name=김은미, action=DRAFT
      - activities[0]: name=김은미, type=DRAFT
      - 매칭 체크: '김은미' in '김은미' = True
      - 매칭 체크: 'DRAFT' == 'DRAFT' = True
      ✅ 첫 번째 매칭 성공!
        - crawled[0]: name=김은미, action=DRAFT
        ✅ 업데이트: 1451833200000 → 1451876977000

    🎯 업데이트 시도: index=1, name=김태정, action=APPROVAL
      - activities[1]: name=김태정, type=APPROVAL
      - 매칭 체크: '김태정' in '김태정' = True
      - 매칭 체크: 'APPROVAL' == 'APPROVAL' = True
      ✅ 첫 번째 매칭 성공!
        - crawled[0]: name=김은미, action=DRAFT
        - crawled[1]: name=김태정, action=APPROVAL
        ✅ 업데이트: 1451833200000 → 1451900894000

    🎯 업데이트 시도: index=8, name=정주연, action=APPROVAL
      ❌ index 8가 범위 초과 (activ